In [1]:
import pandas as pd
import geopandas as gpd
import os
from osgeo import ogr

**Load REMIND output data**    
The data is read, only results for years between 2005 and 2050 are kept     
Europe is taken, and energy service, energy intensity and final energy are displayed

In [2]:
os.chdir(r"C:\Users\mar.eco\OneDrive - CBS - Copenhagen Business School\Desktop\REMIND")
transport_data = pd.read_csv("REMIND_Transport_Output.txt", delimiter='\t')
transport_data = transport_data.drop(columns={"Model", "Scenario","2055", "2060", "2070", "2080", "2090", "2100", "2110", "2130", "2150"})
transport_data.Region.unique()
Variable = transport_data['Variable'].str.split('|', expand=True)

In [3]:
transport_EU = transport_data.query("Region == 'EU27'")

In [4]:
def split_variable(df):
    variables = df['Variable'].str.split('|', expand=True)
    num_cols =  variables.shape[1]
    new_column_names = [f'Variable_{i}' for i in range(num_cols)]
    df[new_column_names] = variables
    return(df)

In [5]:
transport_EU = split_variable(transport_EU)

C:\Users\mar.eco\AppData\Local\Temp\ipykernel_8364\3017919102.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_names] = variables
C:\Users\mar.eco\AppData\Local\Temp\ipykernel_8364\3017919102.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_names] = variables
C:\Users\mar.eco\AppData\Local\Temp\ipykernel_8364\3017919102.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [6]:
transport_EU.Variable_0.unique()

array(['Preference', 'Logit cost', 'Annual mileage', 'Load factor',
       'Load factor (raw)', 'Energy intensity sales',
       'Energy intensity (raw)',
       'TCO sales Operating costs (total non-fuel)',
       'TCO sales Fuel costs', 'TCO sales Capital costs',
       'Purchase Price', 'Stock', 'Time value costs',
       'Inconvenience costs', 'Policy mask', 'Inconvenience costs (raw)',
       'Energy intensity', 'FE', 'ES', 'Emi',
       'Annualized fleet investments', 'Operating costs fleet',
       'Operating costs fleet (fuel)',
       'Operating costs fleet (total non-fuel)', 'UE', 'Sales',
       'Vintages', 'GDPpc', 'GDP', 'Population'], dtype=object)

In [7]:
energy_service = transport_EU[transport_EU["Variable_0"] == "ES"]
energy_service.Variable_1.unique()

array(['Transport', 'Transport edge', 'Transport with bunkers'],
      dtype=object)

In [8]:
energy_service = energy_service[energy_service["Variable_1"] == "Transport"]
energy_service.shape[0]

201

In [9]:
energy_intensity = transport_EU[transport_EU["Variable_0"] == "Energy intensity"]
energy_intensity.Variable_1.unique()

array(['Transport', 'Transport with bunkers'], dtype=object)

In [10]:
energy_intensity = energy_intensity[energy_intensity["Variable_1"] == "Transport"]
energy_intensity.shape[0]

204

In [11]:
final_energy = transport_EU[transport_EU["Variable_0"] == "FE"]
final_energy.Variable_1.unique()

array(['Transport', 'Transport edge', 'Transport with bunkers'],
      dtype=object)

In [12]:
final_energy = final_energy[final_energy["Variable_1"] == "Transport"]
final_energy.shape[0]

348

In [13]:
energy_service.Variable_2.unique()

array(['Bunkers', 'Freight', 'Pass', 'Pass with bunkers',
       'Freight with bunkers', None, 'BEV', 'FCEV', 'Gases', 'Liquids',
       'Hybrid electric', 'Hydrogen', 'Electric', 'Road', 'Rail'],
      dtype=object)

In [14]:
energy_intensity.Variable_2.unique()

array(['Bunkers', 'Freight', 'Pass', 'Pass with bunkers',
       'Freight with bunkers', None, 'Liquids', 'Electric', 'BEV', 'FCEV',
       'Gases', 'Hydrogen', 'Hybrid electric', 'Road', 'Rail'],
      dtype=object)

In [15]:
final_energy.Variable_2.unique()

array(['Bunkers', 'Freight', 'Pass', 'Pass with bunkers',
       'Freight with bunkers', 'Road', 'Rail'], dtype=object)

In [16]:
passenger_ES = energy_service.query("Variable_2 == 'Pass'")
passenger_ES.shape[0]

113

In [17]:
passenger_ES.Variable_3.unique()

array(['Domestic Aviation', 'Road', 'Rail', 'Non-motorized', 'BEV',
       'Electric', 'FCEV', 'Gases', 'Hybrid electric', 'Hydrogen',
       'Liquids', 'Aviation'], dtype=object)

In [18]:
passenger_ES_domestic = passenger_ES.query("Variable_3 == 'Domestic Aviation'")
passenger_ES_aviation = passenger_ES.query("Variable_3 == 'Aviation'")
passenger_ES_road = passenger_ES.query("Variable_3 == 'Road'")
passenger_ES_rail = passenger_ES.query("Variable_3 == 'Rail'")
passenger_ES_nm = passenger_ES.query("Variable_3 == 'Non-motorized'")

In [19]:
passenger_ES_LDV = passenger_ES_road.query("Variable_4 == 'LDV'")
passenger_ES_LDV.Variable_5.unique()

array(['Four Wheelers', 'Two Wheelers', None, 'BEV', 'FCEV', 'Gases',
       'Hybrid electric', 'Liquids'], dtype=object)

In [20]:
passenger_ES_4w = passenger_ES_LDV.query("Variable_5 == 'Four Wheelers'")
passenger_ES_2w = passenger_ES_LDV.query("Variable_5 == 'Two Wheelers'")
passenger_ES_check = passenger_ES_LDV.query("Variable_5 == 'Hybrid electric'")

In [21]:
passenger_EI = energy_intensity.query("Variable_2 == 'Pass'")
passenger_EI.shape[0]

114

In [22]:
passenger_EI.Variable_3.unique()

array(['Domestic Aviation', 'Road', 'Rail', 'Non-motorized', None, 'BEV',
       'Electric', 'FCEV', 'Gases', 'Hybrid electric', 'Hydrogen',
       'Liquids', 'Aviation'], dtype=object)

In [23]:
passenger_EI_domestic = passenger_EI.query("Variable_3 == 'Domestic Aviation'")
passenger_EI_aviation = passenger_EI.query("Variable_3 == 'Aviation'")
passenger_EI_road = passenger_EI.query("Variable_3 == 'Road'")
passenger_EI_rail = passenger_EI.query("Variable_3 == 'Rail'")
passenger_EI_nm = passenger_EI.query("Variable_3 == 'Non-motorized'")

In [24]:
passenger_FE = final_energy.query("Variable_2 == 'Pass'")
passenger_FE.shape[0]

202

In [25]:
passenger_FE.Variable_3.unique()

array(['Domestic Aviation', 'Road', 'Rail', 'Non-motorized', 'Gases',
       'Liquids', 'Aviation'], dtype=object)

In [26]:
passenger_FE_domestic = passenger_FE.query("Variable_3 == 'Domestic Aviation'")
passenger_FE_aviation = passenger_FE.query("Variable_3 == 'Aviation'")
passenger_FE_road = passenger_FE.query("Variable_3 == 'Road'")
passenger_FE_rail = passenger_FE.query("Variable_3 == 'Rail'")
passenger_FE_nm = passenger_FE.query("Variable_3 == 'Non-motorized'")

passenger_FE_gases = passenger_FE.query("Variable_3 == 'Gases'")
passenger_FE_liquids = passenger_FE.query("Variable_3 == 'Liquids'")
passenger_FE_bus = passenger_FE.query("Variable_4 == 'Bus'")
passenger_FE_LDV = passenger_FE.query("Variable_4 == 'LDV'")

In [31]:
passenger_FE_LDV

,Region,Variable,Unit,2005,2010,2015,2020,2025,2030,2035,...,Variable_3,Variable_4,Variable_5,Variable_6,Variable_7,Variable_8,Variable_9,Variable_10,Variable_11,Variable_12
84106,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Large...,EJ/yr,0,0.00141399951180402,0.0015637963461118,0.00128704954258447,0.00101026671140626,0.00135940609154167,0.00192094107071838,...,Road,LDV,Four Wheelers,Large,Large Car and SUV,Gases,Biomass,None,None,None
84131,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Large...,EJ/yr,0.0341449936902262,0.0299978853436819,0.0203272254821719,0.0107177798529844,0.0126340889614524,0.0166438276103832,0.0155718399954911,...,Road,LDV,Four Wheelers,Large,Large Car and SUV,Gases,Fossil,None,None,None
84156,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Large...,EJ/yr,0,0,0,0,0,0,"3,36E+06",...,Road,LDV,Four Wheelers,Large,Large Car and SUV,Gases,Hydrogen,None,None,None
84181,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Large...,EJ/yr,0.0181832550953275,0.0372279115066964,0.0561907635435207,0.083592104241265,0.0897276936368192,0.0896000084105853,0.0696598140773417,...,Road,LDV,Four Wheelers,Large,Large Car and SUV,Liquids,Biomass,None,None,None
84206,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Large...,EJ/yr,14.811.340.689.372,154.746.844.354.651,151.046.216.869.349,144.092.634.304.131,120.006.966.775.638,0.898096924272842,0.518968509542041,...,Road,LDV,Four Wheelers,Large,Large Car and SUV,Liquids,Fossil,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127804,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Small...,EJ/yr,0.0191577982888965,0.0212517856229731,0.0165946252402293,0.0103673779985115,0.0211532678926205,0.0364281833631368,0.0373045847506223,...,Road,LDV,Four Wheelers,Small,Gases,Fossil,None,None,None,None
127828,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Small...,EJ/yr,0,0,0,0,0,0,"7,58E+06",...,Road,LDV,Four Wheelers,Small,Gases,Hydrogen,None,None,None,None
127852,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Small...,EJ/yr,0.0192565037848375,0.0462650863035565,0.0789447777324852,0.127350737647219,0.1324090284324,0.12347802073046,0.089218441248909,...,Road,LDV,Four Wheelers,Small,Liquids,Biomass,None,None,None,None
127876,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Small...,EJ/yr,216.818.991.008.636,227.465.638.816.496,221.368.857.267.704,211.736.921.424.868,183.523.165.025.722,143.066.064.545.547,0.831152333479445,...,Road,LDV,Four Wheelers,Small,Liquids,Fossil,None,None,None,None


In [28]:
passenger_FE_LDV = passenger_FE_road.query("Variable_4 == 'LDV'")
passenger_FE_LDV.Variable_5.unique()

array(['Four Wheelers', 'Two Wheelers', None, 'Electricity', 'Gases',
       'Hydrogen', 'Liquids'], dtype=object)

In [32]:
passenger_FE_4w = passenger_FE_LDV.query("Variable_5 == 'Four Wheelers'")
passenger_FE_2w = passenger_FE_LDV.query("Variable_5 == 'Two Wheelers'")
passenger_FE_check = passenger_FE_LDV.query("Variable_5 == 'Liquids'")
passenger_FE_large = passenger_FE_LDV.query("Variable_6 == 'Large'")
passenger_FE_medium = passenger_FE_LDV.query("Variable_6 == 'Medium'")
passenger_FE_small = passenger_FE_LDV.query("Variable_6 == 'Small'")
passenger_FE_4w

,Region,Variable,Unit,2005,2010,2015,2020,2025,2030,2035,...,Variable_3,Variable_4,Variable_5,Variable_6,Variable_7,Variable_8,Variable_9,Variable_10,Variable_11,Variable_12
84106,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Large...,EJ/yr,0,0.00141399951180402,0.0015637963461118,0.00128704954258447,0.00101026671140626,0.00135940609154167,0.00192094107071838,...,Road,LDV,Four Wheelers,Large,Large Car and SUV,Gases,Biomass,None,None,None
84131,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Large...,EJ/yr,0.0341449936902262,0.0299978853436819,0.0203272254821719,0.0107177798529844,0.0126340889614524,0.0166438276103832,0.0155718399954911,...,Road,LDV,Four Wheelers,Large,Large Car and SUV,Gases,Fossil,None,None,None
84156,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Large...,EJ/yr,0,0,0,0,0,0,"3,36E+06",...,Road,LDV,Four Wheelers,Large,Large Car and SUV,Gases,Hydrogen,None,None,None
84181,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Large...,EJ/yr,0.0181832550953275,0.0372279115066964,0.0561907635435207,0.083592104241265,0.0897276936368192,0.0896000084105853,0.0696598140773417,...,Road,LDV,Four Wheelers,Large,Large Car and SUV,Liquids,Biomass,None,None,None
84206,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Large...,EJ/yr,14.811.340.689.372,154.746.844.354.651,151.046.216.869.349,144.092.634.304.131,120.006.966.775.638,0.898096924272842,0.518968509542041,...,Road,LDV,Four Wheelers,Large,Large Car and SUV,Liquids,Fossil,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127804,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Small...,EJ/yr,0.0191577982888965,0.0212517856229731,0.0165946252402293,0.0103673779985115,0.0211532678926205,0.0364281833631368,0.0373045847506223,...,Road,LDV,Four Wheelers,Small,Gases,Fossil,None,None,None,None
127828,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Small...,EJ/yr,0,0,0,0,0,0,"7,58E+06",...,Road,LDV,Four Wheelers,Small,Gases,Hydrogen,None,None,None,None
127852,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Small...,EJ/yr,0.0192565037848375,0.0462650863035565,0.0789447777324852,0.127350737647219,0.1324090284324,0.12347802073046,0.089218441248909,...,Road,LDV,Four Wheelers,Small,Liquids,Biomass,None,None,None,None
127876,EU27,FE|Transport|Pass|Road|LDV|Four Wheelers|Small...,EJ/yr,216.818.991.008.636,227.465.638.816.496,221.368.857.267.704,211.736.921.424.868,183.523.165.025.722,143.066.064.545.547,0.831152333479445,...,Road,LDV,Four Wheelers,Small,Liquids,Fossil,None,None,None,None


**Comparing data from REMINd and Eurostat**    
Find proxy from Eurostat to agregate by country the values of REMIND output

In [30]:
file_path = 'avia_panc.xlsx'
passenger_domestic_euro = pd.read_excel(file_path, sheet_name='Sheet 1')
passenger_domestic_euro 

FileNotFoundError: [Errno 2] No such file or directory: 'avia_panc.xlsx'

In [ ]:
file_path_compr = r"C:\Users\mar.eco\Desktop\REMIND\estat_avia_panc.tsv.gz"
passenger_domestic_euro_compr = pd.read_csv(file_path_compr, compression='gzip', delimiter='\t')
print(passenger_domestic_euro_compr.head())

In [ ]:
# Rows in passenger_EI but not in passenger_ES
passenger_EI = passenger_EI.reset_index(drop=True)
passenger_ES = passenger_ES.reset_index(drop=True)
diff_ES_EI = passenger_EI[~passenger_EI.index.isin(passenger_ES.index)]
diff_ES_EI

In [ ]:
missing_check = passenger_ES[passenger_ES['Variable'] == "ES|Transport|Pass|Aviation|Hydrogen"]
missing_check

In [ ]:
# Rows in passenger_FE but not in passenger_EI
passenger_FE = passenger_FE.reset_index(drop=True)
diff_FE_EI = passenger_FE[~passenger_FE.index.isin(passenger_EI.index)]
diff_FE_EI

In [ ]:
missing_check = passenger_EI[passenger_EI['Variable'] == "Energy intensity|Transport|Pass|Rail|HSR"]
missing_check

In [ ]:
freight_ES = energy_service.query("Variable_2 == 'Freight'")
freight_ES.shape[0]

In [ ]:
freight_ES.Variable_3.unique()

In [ ]:
freight_ES_domestic = freight_ES.query("Variable_3 == 'Domestic Shipping'")
freight_ES_road = freight_ES.query("Variable_3 == 'Road'")
freight_ES_rail = freight_ES.query("Variable_3 == 'Rail'")

In [ ]:
freight_EI = energy_intensity.query("Variable_2 == 'Freight'")
freight_EI.shape[0]

In [ ]:
freight_EI.Variable_3.unique()

In [ ]:
freight_EI_domestic = freight_EI.query("Variable_3 == 'Domestic Shipping'")
freight_EI_road = freight_EI.query("Variable_3 == 'Road'")
freight_EI_rail = freight_EI.query("Variable_3 == 'Rail'")

In [ ]:
freight_FE = final_energy.query("Variable_2 == 'Freight'")
freight_FE.shape[0]

In [ ]:
freight_FE.Variable_3.unique()

In [ ]:
freight_FE_domestic = freight_FE.query("Variable_3 == 'Domestic Shipping'")
freight_FE_road = freight_FE.query("Variable_3 == 'Road'")
freight_FE_rail = freight_FE.query("Variable_3 == 'Rail'")

freight_FE_unique = freight_FE.Variable_3.unique()
freight_FE_unique

In [ ]:
bunkers = energy_service.query("Variable_2 == 'Bunkers'")
bunkers_all = energy_service[energy_service['Variable'].str.contains("Bunkers")]
bunkers_all

In [ ]:
bunkers = final_energy.query("Variable_2 == 'Bunkers'")
bunkers_all = final_energy[final_energy['Variable'].str.contains("Bunkers")]
bunkers_all

In [ ]:
Bunkers_all = final_energy[final_energy['Variable'].str.contains("Domestic")]
Bunkers_all

In [ ]:
passenger = final_energy.query("Variable_2 == 'Pass'")
passenger_str = final_energy[final_energy["Variable"].str.contains("Pass")]
# Rows in passenger_str but not in passenger
difference_str_not_in_passenger = passenger_str[~passenger_str.index.isin(passenger.index)]
difference_str_not_in_passenger

In [ ]:
freight = final_energy.query("Variable_2 == 'Freight'")
# bunkers = final_energy.query("Variable_2 == 'Bunkers'")
# Tqke only the domestic shipping and aviation instead of bunkers
domestic_shipping = final_energy[final_energy['Variable'].str.contains("Domestic Shipping")]
domestic_aviation = final_energy[final_energy['Variable'].str.contains("Domestic Aviation")]
bunkers = pd.concat([domestic_shipping, domestic_aviation])
bunkers

In [ ]:
freight_road = freight.query("Variable_3 == 'Road'")
freight_domestic_shipping = freight.query("Variable_3 == 'Domestic Shipping'")
freight_rail = freight.query( "Variable_3 == 'Rail'")

In [ ]:
pass_road = passenger.query("Variable_3 == 'Road'")
pass_rail = passenger.query("Variable_3 == 'Rail'")
pass_domestic_aviation = passenger.query("Variable_3 == 'Domestic Aviation'")
pass_aviation = passenger.query("Variable_3 == 'Aviation'")
domestic_pass_aviation = passenger.query("Variable_3 == 'Domestic Aviation'")
pass_non_motorized = passenger.query("Variable_3 == 'Non-motorized'")


##### Aviation
Domestic passenger aviation can be distributed based on Eurostat: National air passenger transport by reporting country (avia_panc)
Bunkers and aviation - International aviation can be distributed badsed on International intra-EU adn extra-EU air passenger transport by reporting country and EU partner country (avia_paincc) & (avia_paexcc)

Other option: Aggregate and get share of domestic and "international" from the 2024 and then use this on total share of passengers on data set: Air transport of passengers by country (yearly data) (ttr00012)



#### Maritime 
International shipping and domestic shipping is distributed on total goods handled in each country. 

Other option would be to distribute these on inside and outside the EU shipped goods. However, domestic is exclusive to maritime transport in that country.  
Domestic shipping is bigger by 3 times than internationla shipping. Needs to be considered. Ask if this can be better approximiated by transport within the EU. 

#### Railway 
Railway transport measurement - passenger - passenger transported. Missing data on Beligium, Hungary, Netherlands, Poland, UK, Montenegro. Data for Poland in data set Rail transport of passenger. Approximatingf rail transport over people? Finding a relationship between passengers transported to people? Average of pkm/p and then scale up to country size. 
Railway transport measurement - goods - goods transported. Missing data on Belgium and Greece. Needs to be approximated, Averages? Less data in the overall dataset. 

In [ ]:
freight_domestic_shipping

In [ ]:
bunkers

In [ ]:
domestic_pass_aviation

In [ ]:
pass_aviation

In [ ]:
passenger.Variable_3.unique()

In [ ]:
energy_service.query("Variable_2 == 'Freight'").Variable_3.unique()

In [ ]:
energy_service.Variable_1.unique()

In [ ]:
energy_service.Variable_2.unique()

In [ ]:
energy_service.Variable_3.unique()

In [ ]:
energy_service.Variable_4.unique()

In [ ]:
energy_service.Variable_5.unique()

In [ ]:
energy_service.Variable_6.unique()

In [ ]:
energy_service.Variable_7.unique()

In [ ]:
energy_service.Variable_8.unique()